In [ ]:
# This script gets the video length and caption ids for the videos in the video list

In [1]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
from google.auth.transport.requests import Request as AuthRequest
from googleapiclient.discovery import build
import httplib2
from pprint import pprint
from datetime import timedelta
import re
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Access API key
yt_api_key = os.getenv("YOUTUBE_API_KEY")


In [2]:
# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=yt_api_key)

In [8]:
### LOAD ###
try:
    videos_with_captions = pd.read_csv('../resources/video_list/videos_with_details.csv', index_col=0)
    print('Succcessfully loaded videos with captions')
    display(videos_with_captions.head(10))
    display(videos_with_captions.tail(10))
    loaded = videos_with_captions.copy()

except:
    videos_df = pd.read_csv('../resources/video_list/videos.csv', index_col=0)
    print('No file found with caption details. Loaded base videos df')
    display(videos_df.head())
    loaded = videos_df.copy()

if 'video_id' in loaded.columns:
    loaded.set_index('video_id', inplace=True)


### CLEAN ###
# Check for duplicate video IDs
duplicate_indices = loaded.index[loaded.index.duplicated()]

# Print the number of duplicate videos
print("Number of duplicate videos:", len(duplicate_indices))

# Print the titles of the duplicate videos
print("Titles of duplicate videos:")
print(loaded.loc[duplicate_indices, 'title'])

# Drop duplicate rows based on index
loaded = loaded[~loaded.index.duplicated()]

# Print the number of dropped videos
print("Dropped videos:", len(duplicate_indices))

Succcessfully loaded videos with captions


,playlist,title,transcribed,length,caption_id
video_id,,,,,
E-aoo7Gl2FQ,52_weeks,How to set up a reef tank | 52 Weeks of Reefing,False,0:01:28,AUieDaanATJE3B_sY3g0XdT4d529fpUTwWeQoRNi8_RNyM...
fKEXNIhomGs,52_weeks,Week 1: Our Best Reef Tank Build Yet | 52 Week...,False,0:02:51,AUieDaZUPwzNfRgZ8pkyQVjAyPoKqobPj2n8FaNwOUqe
OxZ_hJjXwj8,52_weeks,Week 2: Unveiling the tank and custom built st...,False,0:17:38,AUieDaaGEEmWfrg14IXf7M_i6ZmxAgaPbdif7ba6YD67
z6foHVHg1Rw,52_weeks,"Week 3: Sumps - What do they do, and which sho...",False,0:15:11,AUieDaaJ5tS6S0lWQ3n0iLGnQupq2Vc6geBy_kXYiyl5
tppr8V13h5U,52_weeks,Week 4: Planning a Safer Tank with Redundancy ...,False,0:17:34,AUieDabkVqpY2TKn0EG-plpyrFZjHJVCdSiOFCDIE2mY
q1tAnhIGpgA,52_weeks,"Week 5: Plumbing, Overflows, and Return Pumps ...",False,0:19:02,AUieDabKpBko1hu1AwBcvEfjCvJJbbgDqif98oBvI61f2Q...
1rER7e7zTxU,52_weeks,"Week 6: Wiring Your Reef Tank, Everything You ...",False,0:09:31,AUieDaYleDoSmw_Pj_Uga3E6GU56wMcU93wtei59boXGdX...
SMq-vk1mnZw,52_weeks,Week 7: How to Produce the Best Water Possible...,False,0:19:06,AUieDaas5UnHEaum_T4A0EAOKGj8afPQUHL_RKz0RMFd
8vrSO8WLs_M,52_weeks,Week 8: Tank Temperature. Finding a Way to Tru...,False,0:14:53,AUieDabmlIYchPOYyPPocrtLsU54xg2kzH-cPfXyJ-Sh-r...


,playlist,title,transcribed,length,caption_id
video_id,,,,,
3h0vR6ltf-s,nutrients,Nutrients vs Nutrition? A Distinction That Cha...,False,NaN,NaN
YMxx6CAH3KU,nutrients,Ocean vs. Reef Aquarium Parameters: A Nitrate ...,False,NaN,NaN
eRz4kdlBIbY,nutrients,A Magic Ratio? Trailblazing Pest Solutions. Ni...,False,NaN,NaN
qr0Wufsw5fs,nutrients,5:1 or 20:1? Fish Food Nutrient Input. The Sol...,False,NaN,NaN
YsJF2q0pS0U,nutrients,The New Reefer / Mentor Paradox. Reef Tank Sta...,False,NaN,NaN
ZiPJiSKkVqA,nutrients,"My Beliefs: Nutrients, 17 Years, and How I App...",False,NaN,NaN
IqLgZDp2Wzo,nutrients,A Fish First Approach to Optimizing Fish Food ...,False,NaN,NaN
lMzrWkBmX4g,nutrients,A Challenge: Three Effective Filters for Organ...,False,NaN,NaN
I7_Rm39aOwc,nutrients,Are Water Changes Worth It? A Hammer Solution ...,False,NaN,NaN


Number of duplicate videos: 1
Titles of duplicate videos:
video_id
3OmI1WNanjA    Week 10: 17 Questions For Success With Clams i...
3OmI1WNanjA    Week 10: 17 Questions For Success With Clams i...
Name: title, dtype: object
Dropped videos: 1


In [6]:
# Function for video details
def get_video_details(video_id):
    details = []

    # Retrieve video details
    request = youtube.videos().list(
        part="snippet,contentDetails",  # Include 'contentDetails' to get video duration
        id=video_id
    )
    
    # Create an HTTP instance
    http = httplib2.Http()
    headers = {'referer': 'https://youtube.com'}

    # Execute the request
    response, content = http.request(request.uri, method=request.method, body=request.body, headers=headers)
    response_data = json.loads(content)

    # Extract video details from the response
    for item in response_data['items']:
        duration_str = item['contentDetails']['duration']  # Duration is provided in ISO 8601 format
        video_length = parse_duration(duration_str)
        details.append({'video_id': video_id, 'length': str(video_length)})

    return details


# Function to parse duration string into timedelta object
def parse_duration(duration_str):
    # Regular expression to extract hours, minutes, and seconds
    pattern = r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?'
    match = re.match(pattern, duration_str)
    
    # Extract hours, minutes, and seconds
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0
    
    # Create a timedelta object with the parsed hours, minutes, and seconds
    duration = timedelta(hours=hours, minutes=minutes, seconds=seconds)
    
    return duration


# Function for caption ids
def get_caption_ids(video_id):
    captions = []

    # Retrieve captions for the video
    request = youtube.captions().list(
        part="snippet",
        videoId=video_id
    )
    # Create an HTTP instance
    http = httplib2.Http()
    headers = {'referer': 'https://youtube.com'}

    try:
        # Execute the request
        response, content = http.request(request.uri, method=request.method, body=request.body, headers=headers)
        response_data = json.loads(content)
        # pprint(response_data)

        # Extract captions from the response
        for item in response_data['items']:
            caption_id = item['id']
            captions.append({'id': caption_id})
    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"Failed video ID: {video_id}")

    return captions


In [7]:
### MAIN ###

# Test Frame
test_df = loaded.head(10)

#Set environment variable -- dev is for testing the code against the API in small batches to keep quota usage down
production = True

if production == True:
    df = loaded
    environment = 'production'
elif production == False:
    df = test_df.copy()
    environment = 'test'
else:
    print('Error: please set environment')

print(f'Working DataFrame currently set as: ---{environment.upper()}---')

df = df.loc[:, ~df.columns.str.contains('Unnamed')] # Drop any previously generated index duplicates
df.head()



Working DataFrame currently set as: ---PRODUCTION---


,playlist,title,transcribed,length,caption_id
video_id,,,,,
E-aoo7Gl2FQ,52_weeks,How to set up a reef tank | 52 Weeks of Reefing,False,0:01:28,AUieDaanATJE3B_sY3g0XdT4d529fpUTwWeQoRNi8_RNyM...
fKEXNIhomGs,52_weeks,Week 1: Our Best Reef Tank Build Yet | 52 Week...,False,0:02:51,AUieDaZUPwzNfRgZ8pkyQVjAyPoKqobPj2n8FaNwOUqe
OxZ_hJjXwj8,52_weeks,Week 2: Unveiling the tank and custom built st...,False,0:17:38,AUieDaaGEEmWfrg14IXf7M_i6ZmxAgaPbdif7ba6YD67
z6foHVHg1Rw,52_weeks,"Week 3: Sumps - What do they do, and which sho...",False,0:15:11,AUieDaaJ5tS6S0lWQ3n0iLGnQupq2Vc6geBy_kXYiyl5
tppr8V13h5U,52_weeks,Week 4: Planning a Safer Tank with Redundancy ...,False,0:17:34,AUieDabkVqpY2TKn0EG-plpyrFZjHJVCdSiOFCDIE2mY


In [63]:
for index, row in df.iterrows():
    if pd.isnull(row['length']) or pd.isnull(row['caption_id']):  # Check if either length or caption_id is null
        video_id = row['video_id']
        
        # Get video details if length is null
        if pd.isnull(row['length']):
            video_details = get_video_details(video_id)
            if video_details:
                video_length = video_details[0]['length']
                df.loc[index, 'length'] = video_length

        # Get captions if caption_id is null
        if pd.isnull(row['caption_id']):
            captions = get_caption_ids(video_id)
            if captions:
                caption_id = captions[0]['id']
                df.loc[index, 'caption_id'] = caption_id


An error occurred: 'items'
Failed video ID: NWmjpqX8fy8
An error occurred: 'items'
Failed video ID: fs0lLsRSblw
An error occurred: 'items'
Failed video ID: 4lJZCZXE2yo
An error occurred: 'items'
Failed video ID: aM0EEW3ZgUU
An error occurred: 'items'
Failed video ID: S5nha40KdlQ
An error occurred: 'items'
Failed video ID: -f6rRjsnO7w
An error occurred: 'items'
Failed video ID: P9jsNTyeGNk
An error occurred: 'items'
Failed video ID: CqWod-IvqVA
An error occurred: 'items'
Failed video ID: zfcCxzJVqHg


KeyError: 'items'

In [8]:
display(df.tail(30))

,playlist,title,transcribed,length,caption_id
video_id,,,,,
OcxRyucO630,interviews,Will It Change? 6 Things the Industry Can Impr...,False,0:36:06,AUieDaY0nnyGuxNs4aTePTt3ckHk7ZGhqvIk_zPEdu3Yd4...
67nCFUwHgsM,interviews,Better Than Carbon? 10 Tips EVERY Reefer Shoul...,False,0:42:13,AUieDabGhcI-SdnF3twdw7_kuXRe0_3Pb46kZXPfeEIDeo...
BPeEf69NFZ0,interviews,Vic’s Top 10 Things the Reefing Hobby Should S...,False,0:26:39,AUieDaax7SIei63ujNRSaSpBXFEFcrEI4xL85WMlUD08Kj...
xxO6K4yU0eU,interviews,Victor’s 13 Secrets to Running a Successful Co...,False,0:27:01,AUieDaYThG7TzP6T6r9q6D8UlSpxpa4YoiQ1Ft80O7EH8Z...
ez96AVbRik8,interviews,Victor’s 8 Coral Coloration Tips to Make Your ...,False,0:33:30,AUieDaaXij1JOi4oAOdYNV8Ki33WBIfh7ycXkdZVhE6qS9...
O5bgq1jwSfo,interviews,Top 10 Things Victor Wishes EVERY Reefer Knew!,False,0:56:33,AUieDaYP2dR8wX6XNinl5ya8ZKMkO7iKDHhChOaw2G6iEZ...
-f6rRjsnO7w,MACNA,"Calcium & Alkalinity Supplementation, Tropic M...",False,0:48:06,NaN
P9jsNTyeGNk,MACNA,"Spawning Scolymia, LPS and Potential for Susta...",False,0:44:34,NaN
CqWod-IvqVA,MACNA,"The Importance of Plankton, Where to Get It & ...",False,0:56:07,NaN


In [9]:
null_length = df['length'].isnull().value_counts()
print(null_length)
null_caption = df['caption_id'].isnull().value_counts()
print(null_caption)


length
False    249
True      20
Name: count, dtype: int64
caption_id
False    240
True      29
Name: count, dtype: int64


In [10]:
complete = df[df['length'].notnull() & df['caption_id'].notnull()]
display(complete)

stragglers = df[df['length'].isnull() | df['caption_id'].isnull()]
display(stragglers)

,playlist,title,transcribed,length,caption_id
video_id,,,,,
E-aoo7Gl2FQ,52_weeks,How to set up a reef tank | 52 Weeks of Reefing,False,0:01:28,AUieDaanATJE3B_sY3g0XdT4d529fpUTwWeQoRNi8_RNyM...
fKEXNIhomGs,52_weeks,Week 1: Our Best Reef Tank Build Yet | 52 Week...,False,0:02:51,AUieDaZUPwzNfRgZ8pkyQVjAyPoKqobPj2n8FaNwOUqe
OxZ_hJjXwj8,52_weeks,Week 2: Unveiling the tank and custom built st...,False,0:17:38,AUieDaaGEEmWfrg14IXf7M_i6ZmxAgaPbdif7ba6YD67
z6foHVHg1Rw,52_weeks,"Week 3: Sumps - What do they do, and which sho...",False,0:15:11,AUieDaaJ5tS6S0lWQ3n0iLGnQupq2Vc6geBy_kXYiyl5
tppr8V13h5U,52_weeks,Week 4: Planning a Safer Tank with Redundancy ...,False,0:17:34,AUieDabkVqpY2TKn0EG-plpyrFZjHJVCdSiOFCDIE2mY
...,...,...,...,...,...
67nCFUwHgsM,interviews,Better Than Carbon? 10 Tips EVERY Reefer Shoul...,False,0:42:13,AUieDabGhcI-SdnF3twdw7_kuXRe0_3Pb46kZXPfeEIDeo...
BPeEf69NFZ0,interviews,Vic’s Top 10 Things the Reefing Hobby Should S...,False,0:26:39,AUieDaax7SIei63ujNRSaSpBXFEFcrEI4xL85WMlUD08Kj...
xxO6K4yU0eU,interviews,Victor’s 13 Secrets to Running a Successful Co...,False,0:27:01,AUieDaYThG7TzP6T6r9q6D8UlSpxpa4YoiQ1Ft80O7EH8Z...


,playlist,title,transcribed,length,caption_id
video_id,,,,,
NWmjpqX8fy8,52_weeks,Update 4: Changing over to the Triton Method. ...,False,0:12:24,NaN
fs0lLsRSblw,52_weeks,The BRS160 Screensaver: 12hrs of Relaxation Wi...,False,11:54:21,NaN
4lJZCZXE2yo,52_FAQ,PAR meter rental: Setting your reef tank LED l...,False,0:02:20,NaN
aM0EEW3ZgUU,52_FAQ,How to change from 2 part dosing to a Calcium ...,False,0:06:52,NaN
S5nha40KdlQ,11_days,Carbon for Sure... GFO Sometimes... Carbon Dos...,False,0:33:55,NaN
-f6rRjsnO7w,MACNA,"Calcium & Alkalinity Supplementation, Tropic M...",False,0:48:06,NaN
P9jsNTyeGNk,MACNA,"Spawning Scolymia, LPS and Potential for Susta...",False,0:44:34,NaN
CqWod-IvqVA,MACNA,"The Importance of Plankton, Where to Get It & ...",False,0:56:07,NaN
zfcCxzJVqHg,MACNA,ICP & Additives: How to Correct Your Inadequac...,False,0:41:14,NaN


In [11]:
### Save outputs
df.to_csv('../resources/video_list/videos_with_details.csv') 
stragglers.to_csv('../resources/video_list/incomplete.csv')
complete.to_csv('../resources/video_list/details_complete.csv')